In [1]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter



  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-toi4q406
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-toi4q406
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-qt70wds5/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpozx82lpu".


In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)



Mounted at /content/drive/


/tmp/tmps0zs7tzo/dotProduct/main.cu(99): error: argument of type "float" is incompatible with parameter of type "void *"
      cudaMemcpy(finalResult, d_finalResult, sizeof(float), cudaMemcpyDeviceToHost);
                 ^

/tmp/tmps0zs7tzo/dotProduct/main.cu(106): error: expression must have pointer-to-object type but it has type "float"
          f += finalResult[i];
               ^

/tmp/tmps0zs7tzo/dotProduct/main.cu(117): warning #549-D: variable "d_partialSums" is used before its value is set
      cudaFree(d_partialSums);
               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

2 errors detected in the compilation of "/tmp/tmps0zs7tzo/dotProduct/main.cu".



In [3]:
%%cuda_group_save -g "parseImage" -n "main.cu"

#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <cstring>
#include <cuda_runtime.h>


#define THREADS 32


#define IMAGESIZE 784  // Define before using IMAGESIZE

struct TrainingSample {
    int label;                  // Image label (0-9)
    float image[IMAGESIZE];    // Flattened image data
};

// CUDA kernel to compute Euclidean distances
__global__ void computeEuclideanDistances(const float* d_images, const float* d_testImage, float* d_distances, int num_samples) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < num_samples) {
        float sum = 0.0f;
        for (int i = 0; i < IMAGESIZE; ++i) {
            float diff = d_images[idx * IMAGESIZE + i] - d_testImage[i];
            sum += diff * diff;
        }
        d_distances[idx] = sqrtf(sum);
    }
}

bool loadData(const std::string& filename, std::vector<TrainingSample>& samples) {
    std::ifstream file(filename, std::ios::binary);
    if (!file) {
        std::cerr << "Unable to open file " << filename << std::endl;
        return false;
    }

    // Read number of samples
    int num_samples = 0;
    file.read(reinterpret_cast<char*>(&num_samples), sizeof(int));
    if (!file) {
        std::cerr << "Error reading number of samples." << std::endl;
        return false;
    }

    samples.resize(num_samples);

    // Read each sample
    for (int i = 0; i < num_samples; ++i) {
        // Read label
        file.read(reinterpret_cast<char*>(&samples[i].label), sizeof(int));
        if (!file) {
            std::cerr << "Error reading label for sample " << i << std::endl;
            return false;
        }

        // Read image data
        file.read(reinterpret_cast<char*>(samples[i].image), sizeof(float) * IMAGESIZE);
        if (!file) {
            std::cerr << "Error reading image data for sample " << i << std::endl;
            return false;
        }
    }

    file.close();
    return true;
}




int main() {
    std::vector<TrainingSample> train_samples;
    std::vector<TrainingSample> test_samples;

    // Load training data
    if (!loadData("/content/drive/MyDrive/GPU_knn/train_mnist.bin", train_samples)) {
        return -1;  // Exit if data loading fails
    }

    std::cout << "Successfully loaded " << train_samples.size() << " training samples." << std::endl;

    // Load testing data
    if (!loadData("/content/drive/MyDrive/GPU_knn/test_mnist.bin", test_samples)) {
        return -1;  // Exit if data loading fails
    }

    std::cout << "Successfully loaded " << test_samples.size() << " testing samples." << std::endl;

    int num_trainsamples = train_samples.size();
    int num_testsamples = test_samples.size();

    // Allocate host arrays for training data
    int* h_train_labels = new int[num_trainsamples];
    float* h_train_images = new float[num_trainsamples * IMAGESIZE];

    // Transfer data from train_samples vector to host arrays
    for (int i = 0; i < num_trainsamples; ++i) {
        h_train_labels[i] = train_samples[i].label;
        std::memcpy(&h_train_images[i * IMAGESIZE], train_samples[i].image, sizeof(float) * IMAGESIZE);
    }

    // Device pointers for training data
    int* d_train_labels = nullptr;
    float* d_train_images = nullptr;

    // Allocate device memory for training data
    cudaError_t err;

    // Allocate memory for training labels
    err = cudaMalloc((void**)&d_train_labels, num_trainsamples * sizeof(int));
    if (err != cudaSuccess) {
        std::cerr << "Failed to allocate device memory for training labels: " << cudaGetErrorString(err) << std::endl;
        delete[] h_train_labels;
        delete[] h_train_images;
        return -1;
    }

    // Allocate memory for training images
    err = cudaMalloc((void**)&d_train_images, num_trainsamples * IMAGESIZE * sizeof(float));
    if (err != cudaSuccess) {
        std::cerr << "Failed to allocate device memory for training images: " << cudaGetErrorString(err) << std::endl;
        cudaFree(d_train_labels);
        delete[] h_train_labels;
        delete[] h_train_images;
        return -1;
    }

    // Copy training labels to device
    err = cudaMemcpy(d_train_labels, h_train_labels, num_trainsamples * sizeof(int), cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        std::cerr << "Failed to copy training labels to device: " << cudaGetErrorString(err) << std::endl;
        cudaFree(d_train_labels);
        cudaFree(d_train_images);
        delete[] h_train_labels;
        delete[] h_train_images;
        return -1;
    }

    // Copy training images to device
    err = cudaMemcpy(d_train_images, h_train_images, num_trainsamples * IMAGESIZE * sizeof(float), cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        std::cerr << "Failed to copy training images to device: " << cudaGetErrorString(err) << std::endl;
        cudaFree(d_train_labels);
        cudaFree(d_train_images);
        delete[] h_train_labels;
        delete[] h_train_images;
        return -1;
    }

    // Free host memory for training data
    delete[] h_train_labels;
    delete[] h_train_images;

    // Now process the test samples
    int k = 10; // Number of nearest neighbors
    int correct_predictions = 0;

    for (int t = 0; t < num_testsamples; ++t) {
        int test_label = test_samples[t].label;
        float* h_test_image = test_samples[t].image;

        // Allocate device memory for test image
        float* d_test_image = nullptr;
        err = cudaMalloc((void**)&d_test_image, IMAGESIZE * sizeof(float));
        if (err != cudaSuccess) {
            std::cerr << "Failed to allocate device memory for test image: " << cudaGetErrorString(err) << std::endl;
            // Free previously allocated device memory
            cudaFree(d_train_labels);
            cudaFree(d_train_images);
            return -1;
        }

        // Copy test image to device
        err = cudaMemcpy(d_test_image, h_test_image, IMAGESIZE * sizeof(float), cudaMemcpyHostToDevice);
        if (err != cudaSuccess) {
            std::cerr << "Failed to copy test image to device: " << cudaGetErrorString(err) << std::endl;
            cudaFree(d_test_image);
            cudaFree(d_train_labels);
            cudaFree(d_train_images);
            return -1;
        }

        // Allocate device memory for distances
        float* d_distances = nullptr;
        err = cudaMalloc((void**)&d_distances, num_trainsamples * sizeof(float));
        if (err != cudaSuccess) {
            std::cerr << "Failed to allocate device memory for distances: " << cudaGetErrorString(err) << std::endl;
            cudaFree(d_test_image);
            cudaFree(d_train_labels);
            cudaFree(d_train_images);
            return -1;
        }

        // Define grid and block dimensions
        int threadsPerBlock = 256;
        int blocksPerGrid = (num_trainsamples + threadsPerBlock - 1) / threadsPerBlock;

        // Launch the kernel to compute Euclidean distances
        computeEuclideanDistances<<<blocksPerGrid, threadsPerBlock>>>(d_train_images, d_test_image, d_distances, num_trainsamples);

        // Check for kernel launch errors
        err = cudaGetLastError();
        if (err != cudaSuccess) {
            std::cerr << "Kernel launch failed: " << cudaGetErrorString(err) << std::endl;
            cudaFree(d_distances);
            cudaFree(d_test_image);
            cudaFree(d_train_labels);
            cudaFree(d_train_images);
            return -1;
        }

        // Allocate host memory for distances
        float* h_distances = new float[num_trainsamples];

        // Copy distances back to host
        err = cudaMemcpy(h_distances, d_distances, num_trainsamples * sizeof(float), cudaMemcpyDeviceToHost);
        if (err != cudaSuccess) {
            std::cerr << "Failed to copy distances to host: " << cudaGetErrorString(err) << std::endl;
            delete[] h_distances;
            cudaFree(d_distances);
            cudaFree(d_test_image);
            cudaFree(d_train_labels);
            cudaFree(d_train_images);
            return -1;
        }

        // Now perform k-NN classification on the host
        std::vector<std::pair<float, int>> distanceLabelPairs(num_trainsamples);

        for (int i = 0; i < num_trainsamples; ++i) {
            distanceLabelPairs[i] = std::make_pair(h_distances[i], train_samples[i].label);
        }

        // Sort the distances
        std::sort(distanceLabelPairs.begin(), distanceLabelPairs.end());

        // Count the frequency of labels among the k nearest neighbors
        std::vector<int> labelCount(10, 0);  // Assuming labels are digits 0-9

        for (int i = 0; i < k; ++i) {
            int neighborLabel = distanceLabelPairs[i].second;
            labelCount[neighborLabel]++;
        }

        // Find the label with the maximum frequency
        int predictedLabel = std::distance(labelCount.begin(), std::max_element(labelCount.begin(), labelCount.end()));

        // Check if prediction is correct
        if (predictedLabel == test_label) {
            correct_predictions++;
        }

        std::cout << "Test Sample " << t << ": Actual Label = " << test_label << ", Predicted Label = " << predictedLabel << std::endl;

        // Free memory for this test sample
        delete[] h_distances;
        cudaFree(d_distances);
        cudaFree(d_test_image);
    }

    // After processing all test samples
    float accuracy = (float)correct_predictions / num_testsamples * 100.0f;
    std::cout << "Accuracy: " << accuracy << "%" << std::endl;

    // Free device memory for training data
    cudaFree(d_train_labels);
    cudaFree(d_train_images);

    return 0;
}


In [ ]:
%cuda_group_run --group "parseImage" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"